In [52]:
import numpy as np
import time
import matplotlib
from qiskit import IBMQ

# Import Qiskit classes
import qiskit 
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, execute, BasicAer
from qiskit.providers.aer import noise
from qiskit.tools.visualization import plot_histogram

# Import measurement calibration functions
from qiskit.ignis.mitigation.measurement import (complete_meas_cal, tensored_meas_cal,
                                                 CompleteMeasFitter, TensoredMeasFitter)

In [2]:
# Load account from disk
provider = IBMQ.load_account()
# List the account currently in the session
print(IBMQ.active_account())
# List all available providers
print(IBMQ.providers())
# List all available providers 
print(provider.backends())

{'token': '65088dc588c36ec78f59b96f03d954a06f733be46c84aaae67c0e42e3cbb7062e6a316fdd71135631ff5e9e53bc40c7b547b5c880228553ec7c8d549d51feb03', 'url': 'https://auth.quantum-computing.ibm.com/api'}
[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>]
[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmqx4') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>, <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>]


In [3]:
# def exp_decomp_u(a, b, q):
#     return u3(a, b+np.pi, -b, q)

In [54]:
qr = QuantumRegister(2, 'q')
cr = ClassicalRegister(2, 'c')
def vqe_ansatz(theta):
    var_circ= QuantumCircuit(qr)
    var_circ.ry(theta,qr)
    return var_circ

In [55]:
vqe_ansatz(0).draw()

In [56]:
simulator = BasicAer.get_backend('qasm_simulator')
sample_shots = 10000

def expval(parameter):
    
    init_state = vqe_ansatz(parameter)
    circ = QuantumCircuit(qr, cr)
    circ.barrier(qr)
    circ.ry(-np.pi/2, qr[0])
    circ.measure(qr[0], cr[0])
    circ.measure(qr[1], cr[1])
    vqe = init_state + circ
    job = execute(vqe, simulator, shots = sample_shots)
    counts = job.result().get_counts()
    P_00 = counts.get('00', 0)/sample_shots
    P_10 = counts.get('10', 0)/sample_shots
    P_01 = counts.get('01', 0)/sample_shots
    P_11 = counts.get('11', 0)/sample_shots
    final_state = np.array([P_00, P_10, P_01, P_11])
    X = np.array([1, -1, 1, -1])
    Z = np.array([1, 1, -1, -1])
    vqe_ans = -1.0421749 + 0.789689*np.dot(X, final_state) + 0.181210*np.dot(Z, final_state)
    return float(vqe_ans)
test_expval = expval(3.36)
print(test_expval)
type(test_expval)

-1.8566195832


float

In [58]:
from scipy.optimize import minimize
initial_guess=[np.pi]

ans = minimize(expval,initial_guess,method='COBYLA')
ans

     fun: -1.851854311
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 19
  status: 1
 success: True
       x: array([3.32919265])